In [ ]:
!pip install pytorch-lightning

# MNIST

In [ ]:
from torchvision import datasets, transforms
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Subset
import torch

In [ ]:
class MnistDatamodule(pl.LightningDataModule):
  def __init__(self, batch_size = 32):
    super().__init__()
    self.batch_size = batch_size
  def setup(self, stage = None):
    transform = transforms.Compose([transforms.ToTensor(), \
                                transforms.Normalize((0), (1))
                                ])
    
    self.train_dataset = datasets.MNIST(root='data/', train=True, download=True, transform=transform)
    self.test_dataset = datasets.MNIST(root='data/', train=False, download=True, transform=transform)


  def train_dataloader(self):
    return  DataLoader(self.train_dataset, batch_size = self.batch_size, shuffle = True)
  def val_dataloader(self):
    return  DataLoader(self.test_dataset, batch_size = self.batch_size, shuffle = False)

In [ ]:
dm = MnistDatamodule()
dm.setup()

In [ ]:
image = dm.train_dataset[0][0]

In [ ]:
image

In [ ]:
image.shape

torch.Size([1, 28, 28])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image.reshape(28,28), cmap="gray")

In [ ]:
from torch.nn import Conv2d

In [ ]:
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch import flatten
import torchmetrics

In [ ]:
class MnistModel(pl.LightningModule):
  def __init__(self, num_classes):
    super().__init__()
    
    self.loss_function = nn.CrossEntropyLoss()

    self.conv1 = nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(4)
    self.maxpool1 = nn.MaxPool2d(2)

    self.conv2 = nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(4)
    self.maxpool2 = nn.MaxPool2d(2)

    self.fc1 = nn.LazyLinear(100)
    self.fc2 = nn.LazyLinear(10)


    self.relu = nn.ReLU()
    self.flat = nn.Flatten()

    self.train_acc = torchmetrics.Accuracy()
    self.val_acc = torchmetrics.Accuracy()

    self.train_macro_f1 = torchmetrics.F1Score(num_classes=num_classes, average='macro')
    self.val_macro_f1 = torchmetrics.F1Score(num_classes=num_classes, average='macro')

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool1(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.maxpool2(x)

    x = self.flat(x)
    x = self.fc1(x)
    x = self.relu(x)

    x = self.fc2(x)

    return x

  def configure_optimizers(self):
    optimizer =  optim.SGD(self.parameters(), lr = 0.01)
    return optimizer

  def training_step(self, train_batch, batch_idx):
    inputs, labels = train_batch


    outputs = self.forward(inputs.float()) 
    loss = self.loss_function(outputs, labels) 

    self.log('train_loss', loss, on_step= True, on_epoch = True)

    outputs = F.softmax(outputs, dim =1)

    self.train_acc(outputs, labels)
    self.log('train_acc', self.train_acc, on_epoch=True, on_step= False)

    self.train_macro_f1(outputs, labels)
    self.log('train_macro_f1', self.train_macro_f1, on_epoch=True, on_step= False)

    return loss

  def validation_step(self, val_batch, batch_idx):
    inputs, labels = val_batch 


    outputs = self.forward(inputs.float())
    loss = self.loss_function(outputs, labels)

    self.log('val_loss', loss,  on_step= True, on_epoch = True)


    outputs = F.softmax(outputs, dim =1)

    self.val_acc(outputs, labels)
    self.log('val_acc', self.val_acc, on_epoch=True, on_step= False)

    self.val_macro_f1(outputs, labels)
    self.log('val_macro_f1', self.val_macro_f1, on_epoch=True, on_step= False)

    return loss

In [ ]:
mnist_model = MnistModel(10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="mnist model")

In [ ]:
trainer = pl.Trainer(gpus=1, logger = logger, max_epochs = 15, log_every_n_steps =1)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(mnist_model, dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/model_summary/model_summary.py:412: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  "A layer with UninitializedParameter was found. "
INFO:pytorch_lightning.callbacks.model_summary:
   | Name           | Type             | Params
-----------------------------------------------------
0  | loss_function  | CrossEntropyLoss | 0     
1  | conv1          | Conv2d           | 40    
2  | bn1            | BatchNorm2d      | 8     
3  | maxpool1       | MaxPool2d        | 0     
4  | conv2          | Conv2d           | 148   
5  | bn2            | BatchNorm2d      | 8     
6  | maxpool2       | MaxPool2d        | 0     
7  | fc1            | LazyLinear       | 0     
8  | fc2            | LazyLinear       | 0     
9  | relu           | ReLU             | 0     
10 | fl

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

Launching TensorBoard...

KeyboardInterrupt: ignored

https://www.kaggle.com/datasets/alessiocorrado99/animals10

In [ ]:
!unzip "/content/data/archive.zip"

unzip:  cannot find or open /content/data/archive.zip, /content/data/archive.zip.zip or /content/data/archive.zip.ZIP.


#### Wczytywanie danych

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

In [ ]:
from torchvision.transforms.transforms import CenterCrop
class AnimalsDatamodule(pl.LightningDataModule):
  def __init__(self, batch_size = 32):
    super().__init__()
    self.batch_size = batch_size
  def setup(self, stage = None):
    transform = transforms.Compose([
                                    transforms.CenterCrop(200),
                                    transforms.ToTensor(), \
                                    transforms.Normalize((0), (1))
                                  ])
    
    dataset = datasets.ImageFolder(root='raw-img/', transform=transform)
    print(len(dataset))
    self.train_dataset, self.val_dataset = random_split(dataset, [ len(dataset)- len(dataset)//4, len(dataset)//4])

  def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle =True)

  def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle = False)

In [ ]:
dm = AnimalsDatamodule()

In [ ]:
dm.setup()

In [ ]:
image = dm.train_dataset[0][0]

In [ ]:
image.shape

In [ ]:
len(dm.train_dataset)

In [ ]:
len(dm.test_dataset)

In [ ]:
class AnimalsModel(pl.LightningModule):
  def __init__(self, num_classes):
    super().__init__()
    
    self.loss_function = nn.CrossEntropyLoss()

    self.conv1 = nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(4)
    self.maxpool1 = nn.MaxPool2d(2)

    self.conv2 = nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(4)
    self.maxpool2 = nn.MaxPool2d(2)

    self.fc1 = nn.LazyLinear(100)
    self.fc2 = nn.LazyLinear(10)

    self.train_acc = torchmetrics.Accuracy()
    self.val_acc = torchmetrics.Accuracy()

    self.train_macro_f1 = torchmetrics.F1Score(num_classes=num_classes, average='macro')
    self.val_macro_f1 = torchmetrics.F1Score(num_classes=num_classes, average='macro')

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.maxpool1(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.maxpol2(x)

    x = self.flat(x)
    x = self.fc1(x)
    x = self.relu(x)
    
    x = self.fc2
    
    return x

  def configure_optimizers(self):
    optimizer =  optim.SGD(self.parameters(), lr = 0.001)
    return optimizer

  def training_step(self, train_batch, batch_idx):
    inputs, labels = train_batch


    outputs = self.forward(inputs.float()) 
    loss = self.loss_function(outputs, labels) 

    self.log('train_loss', loss, on_step= True, on_epoch = True)

    outputs = F.softmax(outputs, dim =1)

    self.train_acc(outputs, labels)
    self.log('train_acc', self.train_acc, on_epoch=True, on_step= False)

    self.train_macro_f1(outputs, labels)
    self.log('train_macro_f1', self.train_macro_f1, on_epoch=True, on_step= False)


    return loss

  def validation_step(self, val_batch, batch_idx):
    pass

In [ ]:
mnist_model = AnimalsModel(10)

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="animals model")

In [ ]:
trainer = pl.Trainer(gpus=1, logger = logger, max_epochs = 15, log_every_n_steps =1)

In [ ]:
trainer.fit(mnist_model, dm)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs